In [18]:
import pandas as pd

from pyqubo import Constraint, Array
import neal
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridBQMSampler,  LeapHybridSampler
import dimod

In [2]:
data = pd.read_excel("data.xlsx")

In [5]:
columns = ["name", "position", "value", "total_points"]

In [6]:
data = data[columns]

In [7]:
df_use = data.sort_values("position").reset_index(drop=True)

In [8]:
for i in range(len(df_use)):
    df_use.loc[i, "variable"] = "x" + str(i)
df_use

,name,position,value,total_points,variable
0,Benjamin White,DEF,56,78,x0
1,Vitalii Mykolenko,DEF,46,78,x1
2,Virgil van Dijk,DEF,63,80,x2
3,Kieran Trippier,DEF,67,102,x3
4,Oleksandr Zinchenko,DEF,52,85,x4
5,Trent Alexander-Arnold,DEF,85,99,x5
6,William Saliba,DEF,56,84,x6
7,Gabriel dos Santos Magalhães,DEF,50,76,x7
8,James Tarkowski,DEF,46,78,x8
9,Pedro Porro,DEF,59,85,x9


In [9]:
columns = ["variable","name", "position", "value", "total_points"]

In [15]:
total_points = df_use["total_points"].to_list()

In [10]:
df_use = df_use[columns]

In [12]:
num_var = 38
ns = 10

In [13]:
x = Array.create('x', shape=num_var, vartype='BINARY')
a = Array.create('a', shape=ns, vartype='BINARY')

In [14]:
x

Array([Binary('x[0]'), Binary('x[1]'), Binary('x[2]'), Binary('x[3]'), Binary('x[4]'), Binary('x[5]'), Binary('x[6]'), Binary('x[7]'), Binary('x[8]'), Binary('x[9]'), Binary('x[10]'), Binary('x[11]'), Binary('x[12]'), Binary('x[13]'), Binary('x[14]'), Binary('x[15]'), Binary('x[16]'), Binary('x[17]'), Binary('x[18]'), Binary('x[19]'), Binary('x[20]'), Binary('x[21]'), Binary('x[22]'), Binary('x[23]'), Binary('x[24]'), Binary('x[25]'), Binary('x[26]'), Binary('x[27]'), Binary('x[28]'), Binary('x[29]'), Binary('x[30]'), Binary('x[31]'), Binary('x[32]'), Binary('x[33]'), Binary('x[34]'), Binary('x[35]'), Binary('x[36]'), Binary('x[37]')])

In [ ]:
# Hamiltonian for the objective function
HZ = sum(n * x for x, n in zip(x, ratings))